## CAKE experiment on Movies

In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, average_precision_score, accuracy_score
from dataset import Dataset
from myModel import MyModel, MyDataset
from myExplainers import MyExplainer
from myEvaluation import MyEvaluation
import pickle
from tqdm import tqdm
import datetime
import csv
import warnings
import torch
import tensorflow as tf
from scipy.special import softmax
from helper import print_results
from cake import CAKE

In [3]:
torch.cuda.empty_cache() 

Load model, data, and task

In [4]:
data_path = ''
model_path = 'Trained Models/'
save_path = '/home/myloniko/ethos/Results/MV/'

In [5]:
model_name = 'bert'
existing_rationales = True

In [ ]:
task = 'single_label'
sentence_level = True
labels = 2
model = MyModel(model_path, 'bert_movies', model_name, task, labels, False)
max_sequence_len = model.tokenizer.max_len_single_sentence
tokenizer = model.tokenizer
import torch
torch.cuda.is_available()
model.trainer.model.to('cuda')

In [7]:
mv = Dataset(path = data_path)
x, y, label_names, rationales = mv.load_movies(level='sentence')

/home/myloniko/ethos/dataset.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.rationales = np.array(rationales)


In [8]:
from sklearn.model_selection import train_test_split
existing_rationales = True
indices = np.arange(len(y))
train_texts, test_texts, train_labels, test_labels, _, test_indexes = train_test_split(x, list(y), indices, test_size=.2, random_state=42)
if existing_rationales:
    test_rationales = [rationales[x] for x in test_indexes]
size = (0.1 * len(y)) / len(train_labels)
train_texts, validation_texts, train_labels, validation_labels = train_test_split(list(train_texts), train_labels, test_size=size, random_state=42)
train_texts.append(test_texts[84])
train_labels.append(test_labels[84])
train_texts.append(test_texts[72])
train_labels.append(test_labels[72])
test_texts.pop(84)
test_labels.pop(84)
test_rationales.pop(84)
test_texts.pop(72)
test_labels.pop(72)
test_rationales.pop(72)
test_texts.pop(63)
test_labels.pop(63)
test_rationales.pop(63)

array([1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0.])

In [9]:
test_test_rationales = []
for i in range(len(test_rationales)):
    if (test_labels[i] == 1):
        test_test_rationales.append([[0]*len(test_rationales[i][:-1]),list(test_rationales[i][:-1])])
    else:
        test_test_rationales.append([list(test_rationales[i][:-1]),[0]* len(test_rationales[i][:-1])])

In [ ]:
predictions = []
for test_text in test_texts:
    outputs = model.my_predict(test_text)
    predictions.append(outputs[0])

pred_labels = []
for prediction in predictions:
    pred_labels.append(np.argmax(softmax(prediction)))

def average_precision_wrapper(y, y_pred, view):
    return average_precision_score(y, y_pred.toarray(), average=view)

average_precision_score(test_labels, pred_labels, average='macro'), accuracy_score(test_labels, pred_labels), f1_score(test_labels, pred_labels, average='macro'), f1_score(test_labels, pred_labels, average='binary')

In [11]:
label_names

['NEG', 'POS']

Define descriptions for each label

In [10]:
description=["positive: The positive sentiment in the context of the Movies dataset refers to the overall positive tone of a movie review. It indicates that the reviewer has expressed a favorable opinion of the movie they have watched. Examples of positive sentiment in movie reviews include praising the plot, complimenting the acting or directing, or expressing overall satisfaction with the film. Identifying positive sentiment is an important task in natural language processing and sentiment analysis, as it can provide insights into the public perception of a movie and help businesses gauge customer satisfaction.",
        "negative: The negative sentiment label in the Movies dataset refers to the overall negative tone of a movie review. It indicates that the reviewer has expressed an unfavorable opinion of the movie they have watched. Examples of negative sentiment in movie reviews might include criticism of the plot, acting, or directing, or expressing overall dissatisfaction with the film. Identifying negative sentiment is an important task in sentiment analysis, as it can help businesses understand what aspects of their product or service are not meeting customer expectations, and can provide insights into how they might improve customer satisfaction."]

In [11]:
train_label_arrays=[]
for i in range(0,len(train_labels)):
    train_label_arrays.append([train_labels[i],abs(1-train_labels[i])])

In [12]:
cake = CAKE(model_path = 'Trained Models/bert_movies', tokenizer = tokenizer, label_names = label_names, 
            label_descriptions = description, input_docs = train_texts, input_labels = train_label_arrays, 
            input_docs_test = test_texts)

In [13]:
my_explainers = MyExplainer(label_names, model, True, '‡', cake=cake)

my_evaluators = MyEvaluation(label_names, model.my_predict, True, True)
my_evaluatorsP = MyEvaluation(label_names, model.my_predict, True, False)
evaluation =  {'F':my_evaluators.faithfulness, 'FTP': my_evaluators.faithful_truthfulness_penalty, 
          'NZW': my_evaluators.nzw, 'AUPRC': my_evaluators.auprc}
evaluationP = {'F':my_evaluatorsP.faithfulness, 'FTP': my_evaluatorsP.faithful_truthfulness_penalty, 
          'NZW': my_evaluatorsP.nzw, 'AUPRC': my_evaluators.auprc}

In [16]:
confs = []
for key_emb in [1, 2, 3]:
    for label_emb in [1, 2, "2_doc", 3]:
        for keyphrases in [5, 10, 15, 20]:
            for width in [0, 1, 2, 3]:
                for negatives in [True, False]:
                    confs.append([key_emb, label_emb, keyphrases, width, negatives])
len(confs)

288

In [ ]:
import time
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    file_name = save_path + 'MOVIES_BERT_SENTENCE_CAKE_1_2_1_'+str(now.day) + '_' + str(now.month) + '_' + str(now.year)
    metrics = {'F':[], 'FTP':[], 'NZW':[], 'AUPRC':[]}
    metricsP = {'F':[], 'FTP':[], 'NZW':[], 'AUPRC':[]}
    time_r = []
    for conf in confs:
        time_r.append([])
    techniques = [my_explainers.cake_explain] 
    for ind in tqdm(range(0,len(test_texts))):
        torch.cuda.empty_cache() 
        test_rational = test_test_rationales[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        prediction, _, _ = model.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
    
        interpretations = []
        kk = 0
        for conf in confs:
            if conf[1] == 3:
                my_explainers.cake_conf = [conf[0], conf[1], ind, conf[2], conf[3], conf[4]]
            else:
                my_explainers.cake_conf = [conf[0], conf[1], None, conf[2], conf[3], conf[4]]
            ts = time.time()
            temp = techniques[0](instance, prediction, tokens, mask, _, _)
            temp_tokens = tokens.copy()
            if sentence_level:
                temp_tokens = temp[0].copy()[0]
                temp = temp[1].copy()
            interpretations.append([np.array(i)/np.max(np.abs(i)) if np.max(np.abs(i))!=0 else np.zeros(len(i)) for i in temp])
            time_r[kk].append(time.time()-ts)
            kk = kk + 1
        for metric in metrics.keys():
            evaluated = []
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _, instance, prediction, temp_tokens, _, _, test_rational))
            metrics[metric].append(evaluated)
        my_evaluatorsP.saved_state = my_evaluators.saved_state.copy()
        my_evaluators.clear_states()
        for metric in metrics.keys():
            evaluatedP = []
            for interpretation in interpretations:
                evaluatedP.append(evaluationP[metric](interpretation, _, instance, prediction, temp_tokens, _, _, test_rational))
            metricsP[metric].append(evaluatedP)
        with open(file_name+'(A).pickle', 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'(P).pickle', 'wb') as handle:
            pickle.dump(metricsP, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open(file_name+'_TIME.pickle', 'wb') as handle:
            pickle.dump(time_r, handle, protocol=pickle.HIGHEST_PROTOCOL)
time_r = np.array(time_r)
time_r.mean(axis=1)

In [ ]:
print_results(file_name+'(P)', confs, metricsP, label_names)

# Time Analysis

In [23]:
confs = []
for key_emb in [1, 2, 3]:
    for label_emb in [1, 2, 3]:
        for keyphrases in [5, 10, 15, 20]:
            for width in [0, 1, 2, 3]:
                for negatives in [False]:
                    confs.append([key_emb, label_emb, keyphrases, width, negatives])
len(confs)

144

In [ ]:
import time
from tqdm.notebook import tqdm
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)
    
    now = datetime.datetime.now()
    time_r = []
    for conf in confs:
        time_r.append([])
    techniques = [my_explainers.cake_explain] 
    for ind in tqdm(range(0,10), position=0):
        torch.cuda.empty_cache() 
        test_rational = test_test_rationales[ind]
        instance = test_texts[ind]
        my_evaluators.clear_states()
        my_evaluatorsP.clear_states()
        prediction, _, _ = model.my_predict(instance)
        enc = model.tokenizer([instance,instance], truncation=True, padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
    
        interpretations = []
        kk = 0
        for conf in tqdm(confs, position=1, leave=False):
            #print(conf)
            if conf[1] == 3:
                my_explainers.cake_conf = [conf[0], conf[1], ind, conf[2], conf[3], conf[4]]
            else:
                my_explainers.cake_conf = [conf[0], conf[1], None, conf[2], conf[3], conf[4]]
            ts = time.time()
            temp = techniques[0](instance, prediction, tokens, mask, _, _)
            temp_tokens = tokens.copy()
            if sentence_level:
                temp_tokens = temp[0].copy()[0]
                temp = temp[1].copy()
            aaaa = [np.array(i)/np.max(np.abs(i)) if np.max(np.abs(i))!=0 else np.zeros(len(i)) for i in temp]
            time_r[kk].append(time.time()-ts)
            kk = kk + 1
time_r = np.array(time_r)
time_r.mean(axis=1)

In [43]:
time_r2 = np.array(nt)

In [46]:
list(zip(confs,list(time_r2.mean(axis=1))))

[([1, 1, 5, 0, False], 0.4223729848861694),
 ([1, 1, 5, 1, False], 0.4299570322036743),
 ([1, 1, 5, 2, False], 0.43325784206390383),
 ([1, 1, 5, 3, False], 0.42571249008178713),
 ([1, 1, 10, 0, False], 0.43698935508728026),
 ([1, 1, 10, 1, False], 0.42425272464752195),
 ([1, 1, 10, 2, False], 0.4416362285614014),
 ([1, 1, 10, 3, False], 0.427693247795105),
 ([1, 1, 15, 0, False], 0.4219777822494507),
 ([1, 1, 15, 1, False], 0.42236201763153075),
 ([1, 1, 15, 2, False], 0.4425539970397949),
 ([1, 1, 15, 3, False], 0.4239311933517456),
 ([1, 1, 20, 0, False], 0.42103238105773927),
 ([1, 1, 20, 1, False], 0.4258598566055298),
 ([1, 1, 20, 2, False], 0.42176084518432616),
 ([1, 1, 20, 3, False], 0.4171828031539917),
 ([1, 2, 5, 0, False], 0.42262930870056153),
 ([1, 2, 5, 1, False], 0.41694817543029783),
 ([1, 2, 5, 2, False], 0.432607626914978),
 ([1, 2, 5, 3, False], 0.42190544605255126),
 ([1, 2, 10, 0, False], 0.4257162094116211),
 ([1, 2, 10, 1, False], 0.42602787017822263),
 ([1, 2, 